In [20]:
"""Notebook for preprocessing of input text."""

import torch
import torch.nn as nn
from torch.nn import functional as F

In [3]:
# read file
with open("input.txt", "r") as file:
    text = file.read()

print(f"Dataset length (in characters): {len(text)}")

Dataset length (in characters): 1115394


In [4]:
# preview data
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [5]:
# get characters in text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(f"Characters: {''.join(chars)}\nVocabulary size: {vocab_size}")

Characters: 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
Vocabulary size: 65


In [6]:
# create basic encoder and decoder
stoi = {c: i for i, c in enumerate(chars)}
itos = {i: c for i, c in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda v: "".join([itos[i] for i in v])

print(encode("Hello world!"))
print(decode(encode("Hello world!")))

# this is a very simple encoder/decoder, some advanced, popular ones include:
# - OpenAI's TikToken: https://github.com/openai/tiktoken
# - Google's Sentencepiece: https://github.com/google/sentencepiece

[20, 43, 50, 50, 53, 1, 61, 53, 56, 50, 42, 2]
Hello world!


In [7]:
# change data into torch tensor
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [8]:
# train/test split
split = int(0.9 * len(data))
train_data = data[:split]
val_data = data[split:]

In [9]:
# define block for data loading
BLOCK_SIZE = 8
print(train_data[: BLOCK_SIZE + 1])

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])


In [12]:
# showcase the inputs and targets of a transformer
inputs = train_data[:BLOCK_SIZE]
targets = train_data[1 : BLOCK_SIZE + 1]
for t in range(BLOCK_SIZE):
    context = inputs[: t + 1]
    target = targets[t]
    print(f"For input {context} the target is {target}")

For input tensor([18]) the target is 47
For input tensor([18, 47]) the target is 56
For input tensor([18, 47, 56]) the target is 57
For input tensor([18, 47, 56, 57]) the target is 58
For input tensor([18, 47, 56, 57, 58]) the target is 1
For input tensor([18, 47, 56, 57, 58,  1]) the target is 15
For input tensor([18, 47, 56, 57, 58,  1, 15]) the target is 47
For input tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target is 58


In [16]:
# showcase batches
BATCH_SIZE = 4


def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - BLOCK_SIZE, (BATCH_SIZE,))
    x = torch.stack([data[i : i + BLOCK_SIZE] for i in ix])
    y = torch.stack([data[i + 1 : i + BLOCK_SIZE + 1] for i in ix])
    return x, y


get_batch("train")

(tensor([[ 6,  0, 13, 52, 42,  1, 61, 47],
         [47, 43, 60, 43,  1, 47, 58,  6],
         [56, 39, 63,  1, 58, 46, 43, 43],
         [59, 58,  1, 56, 47, 42, 42, 50]]),
 tensor([[ 0, 13, 52, 42,  1, 61, 47, 58],
         [43, 60, 43,  1, 47, 58,  6,  0],
         [39, 63,  1, 58, 46, 43, 43,  6],
         [58,  1, 56, 47, 42, 42, 50, 47]]))

In [21]:
class BigramLanguageModel(nn.Module):
    # simple model that returns probability of P(c_t|c_t-1)

    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets):
        # returns (Batch, Time, Channels)
        logits = self.token_embedding_table(idx)

        B, T, C = logits.shape
        logits = logits.view(B * T, C)
        targets = targets.view(B * T)
        loss = F.cross_entropy(logits, targets)

        return logits, loss


model = BigramLanguageModel(vocab_size)
logits, loss = model(*get_batch("train"))
print(logits, loss)

tensor([[ 1.1406,  1.9804,  0.4508,  ..., -0.9053, -0.1376,  1.8780],
        [-1.6987,  0.2824, -0.0080,  ...,  1.3287,  0.6979,  1.6363],
        [ 0.0965, -0.8656, -0.0278,  ...,  1.7241, -0.2314, -0.8649],
        ...,
        [ 1.6600, -0.8128, -2.1234,  ..., -0.5305,  1.2260,  1.3510],
        [-0.8510, -0.6517, -0.3343,  ...,  1.2178, -0.3811,  0.2976],
        [ 1.6600, -0.8128, -2.1234,  ..., -0.5305,  1.2260,  1.3510]],
       grad_fn=<ViewBackward0>) tensor(4.4245, grad_fn=<NllLossBackward0>)
